<a href="https://colab.research.google.com/github/anilkumarKanasani/LangchainSideProjects/blob/main/4_Marketing_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [4]:
!pip install langchain -q
!pip install Openai -q
!pip install tiktoken -q
!pip install faiss-cpu -q
!pip install environs -q
!pip install streamlit -q

In [5]:
from environs import Env
env = Env()
# Read .env into os.environ
env.read_env("./env")

## Marketing app

In [9]:
%%writefile app.py
import streamlit as st
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

llm = OpenAI(temperature=.9, model="text-davinci-003")

examples = [
    {
        "query": "What is a mobile?",
        "answer": "A mobile is a magical device that fits in your pocket, like a mini-enchanted playground. It has games, videos, and talking pictures, but be careful, it can turn grown-ups into screen-time monsters too!"
    }, {
        "query": "What are your dreams?",
        "answer": "My dreams are like colorful adventures, where I become a superhero and save the day! I dream of giggles, ice cream parties, and having a pet dragon named Sparkles.."
    }, {
        "query": " What are your ambitions?",
        "answer": "I want to be a super funny comedian, spreading laughter everywhere I go! I also want to be a master cookie baker and a professional blanket fort builder. Being mischievous and sweet is just my bonus superpower!"
    }, {
        "query": "What happens when you get sick?",
        "answer": "When I get sick, it's like a sneaky monster visits. I feel tired, sniffly, and need lots of cuddles. But don't worry, with medicine, rest, and love, I bounce back to being a mischievous sweetheart!"
    }, {
        "query": "WHow much do you love your dad?",
        "answer": "Oh, I love my dad to the moon and back, with sprinkles and unicorns on top! He's my superhero, my partner in silly adventures, and the one who gives the best tickles and hugs!"
    }, {
        "query": "Tell me about your friend?",
        "answer": "My friend is like a sunshine rainbow! We laugh, play, and have magical parties together. They always listen, share their toys, and make me feel special. Friendship is the best adventure!"
    }, {
        "query": "What math means to you?",
        "answer": "Math is like a puzzle game, full of numbers and shapes. It helps me count my toys, build towers, and share treats equally. It's fun and makes my brain sparkle!"
    }, {
        "query": "What is your fear?",
        "answer": "Sometimes I'm scared of thunderstorms and monsters under my bed. But with my teddy bear by my side and lots of cuddles, I feel safe and brave again!"
    }
]

def get_prmpt_templte(numberOfWords,
                      inp_age_option,
                      inp_tasktype_option,
                      inp_userInput):

  example_template = """
  Question: {query}
  Response: {answer}
  """

  example_prompt = PromptTemplate(
      input_variables=["query", "answer"],
      template=example_template
  )


  example_selector = LengthBasedExampleSelector(
      examples=examples,
      example_prompt=example_prompt,
      max_length=numberOfWords
  )

  prefix = """You are a {age_option}, and {tasktype_option} :
  Here are some examples:
  """

  suffix = """
  Question: {userInput}
  Response: """


  new_prompt_template = FewShotPromptTemplate(
      example_selector=example_selector,  # use example_selector instead of examples
      example_prompt=example_prompt,
      prefix=prefix,
      suffix=suffix,
      input_variables=["age_option", "tasktype_option", "userInput"],
      example_separator="\n"
  )

  final_prompt = new_prompt_template.format(age_option=inp_age_option,
                             tasktype_option=inp_tasktype_option,
                             userInput=inp_userInput)

  return final_prompt



#UI Starts here

st.set_page_config(page_title="Marketing Tool",
                    page_icon='✅',
                    layout='centered',
                    initial_sidebar_state='collapsed')
st.header("Hey, How can I help you?")

form_input = st.text_area('Enter text', height=50)

inp_tasktype_option = st.selectbox(
    'Please select the action to be performed?',
    ('Write a sales copy', 'Create a tweet', 'Write a product description'),key=1)

inp_age_option= st.selectbox(
    'For which age group?',
    ('Kid', 'Adult', 'senior Citizen'),key=2)

numberOfWords= st.slider('Words limit', 1, 200, 25)

submit = st.button("Generate")

if submit:
  our_prompt = get_prmpt_templte(numberOfWords,
                                  inp_age_option,
                                  inp_tasktype_option,
                                  form_input)
  st.header("Response : ")
  st.write(llm(our_prompt))


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501